B1: find all the disjoint index groups where r2 is constant whenever j in I^m

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image
import os
import argparse
import time
from tqdm import trange
from tensorboardX import SummaryWriter
from collections import OrderedDict
from torchsummary import summary

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import RetinaMNIST
from medmnist import INFO, Evaluator

In [14]:
info = INFO["retinamnist"]
DataClass = getattr(medmnist, info['python_class'])
data_transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize(mean=[.5], std=[.5])])
train_dataset = DataClass(split='train', transform=data_transform, download=True, as_rgb=True)
train_dataset = train_dataset #.cpu().detach().numpy()
X_train = []
y_train = []
for i in range(len(train_dataset)):
    X_train.append(train_dataset[i][0].cpu().detach().numpy())
    y_train.append(train_dataset[i][1])
print(type(train_dataset))
print(X_train)
X_train = np.array(X_train)
y_train = np.array(y_train)

Using downloaded and verified file: /home/matthijs/.medmnist/retinamnist.npz
<class 'medmnist.dataset.RetinaMNIST'>
[array([[[-0.9843137 , -0.96862745, -0.96862745, ..., -1.        ,
         -1.        , -1.        ],
        [-0.94509804, -1.        , -1.        , ..., -1.        ,
         -1.        , -1.        ],
        [-0.9529412 , -1.        , -1.        , ..., -0.9843137 ,
         -0.99215686, -0.99215686],
        ...,
        [-1.        , -0.99215686, -0.9764706 , ..., -0.99215686,
         -0.99215686, -0.99215686],
        [-0.96862745, -0.99215686, -0.9764706 , ..., -0.99215686,
         -0.99215686, -1.        ],
        [-0.92156863, -0.9843137 , -0.9764706 , ..., -0.99215686,
         -1.        , -1.        ]],

       [[-1.        , -0.9764706 , -0.9764706 , ..., -0.9764706 ,
         -0.9764706 , -0.9764706 ],
        [-0.96862745, -1.        , -1.        , ..., -0.9843137 ,
         -0.99215686, -0.99215686],
        [-0.9607843 , -1.        , -1.        , ...,

In [99]:
from tensorflow.keras.models import Sequential
class CustomModel(Sequential):
    def train_step(self, data):
        global gradients
        global tape
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data
        with tf.GradientTape() as tape:
          y_pred = self(x, training=True)  # Forward pass
          # Compute the loss value
          # (the loss function is configured in `compile()`)
          print(type(y), type(y_pred))
          tape.watch(x)
          tape.watch_variables()
          loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
          trainable_vars = self.trainable_variables
          gradients = tape.gradient(loss, trainable_vars)
        # Compute gradients
        
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    global training_finished
    training_finished = True
    print("hi")


In [100]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
X_train_flatten = X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3]))

model = CustomModel()
model.add(Dense(512, input_shape=(X_train_flatten.shape[1:]), activation='relu'))
model.add(Dense(5, activation='relu'))
model.compile(optimizer="Adam", loss=tf.keras.metrics.sparse_categorical_crossentropy, metrics=["acc"])


In [75]:
EPOCHS = 1
for epoch in range(EPOCHS):
    data = (X_train_flatten, y_train)
    print(data)
    model.train_step(data)

(array([[-0.9843137 , -0.96862745, -0.96862745, ..., -0.9529412 ,
        -0.9372549 , -0.9372549 ],
       [-0.9764706 , -0.9764706 , -0.9764706 , ..., -1.        ,
        -0.99215686, -0.99215686],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -0.99215686, -0.99215686],
       ...,
       [-0.5686275 , -0.8117647 , -0.9764706 , ..., -0.99215686,
        -0.9764706 , -0.96862745],
       [-1.        , -1.        , -1.        , ..., -0.9843137 ,
        -0.9843137 , -0.9843137 ],
       [-0.94509804, -0.96862745, -0.9607843 , ..., -0.94509804,
        -0.94509804, -0.94509804]], dtype=float32), array([[0],
       [0],
       [0],
       ...,
       [2],
       [2],
       [3]]))
<class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


AttributeError: 'tuple' object has no attribute 'rank'

In [101]:
history = model.fit(X_train_flatten[:2], y_train[:2], epochs=10)
print(history.history)

Epoch 1/10
<class 'tensorflow.python.framework.ops.Tensor'> <class 'tensorflow.python.framework.ops.Tensor'>


AttributeError: in user code:

    File "/home/matthijs/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/matthijs/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/matthijs/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/tmp/ipykernel_14502/3457522820.py", line 15, in train_step
        tape.watch_variables()

    AttributeError: 'GradientTape' object has no attribute 'watch_variables'


In [98]:
tape.watch_variables()

AttributeError: 'GradientTape' object has no attribute 'watch_variables'

In [16]:
import tensorflow as tf
X_train_flatten = X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3]))

# define model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=X_train_flatten.shape[1:]),
    tf.keras.layers.Dense(5, activation='relu')
])

# compile the model with crossentropy loss
model.compile(loss=tf.keras.metrics.sparse_categorical_crossentropy, optimizer='adam')

# define callback to get gradients after each epoch
class GradientsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gradients = self.model.optimizer.get_gradients(
            self.model.loss, self.model.trainable_weights)
        print('Gradients:', gradients)

# train the model and get gradients after each epoch
model.fit(X_train_flatten, y_train, epochs=10, callbacks=[GradientsCallback()])

Epoch 1/10
29/34 [========================>.....] - ETA: 0s - loss: 4.0981

TypeError: Failed to convert elements of <function sparse_categorical_crossentropy at 0x7f6b82f49310> to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.

In [18]:
import tensorflow as tf
X_train_flatten = X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3]))

# define model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train_flatten.shape[:1])),
    tf.keras.layers.Dense(5, activation='relu')
])

# define optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# define training loop
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        # compute model predictions and loss
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
        
    # compute gradients
    gradients = tape.gradient(loss, model.trainable_weights)
    
    # update model parameters
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    return loss

# train the model and get gradients after each epoch
for epoch in range(10):
    epoch_loss = 0.0
    for i in range(X_train_flatten.shape[0]):
        x = X_train_flatten[i]
        y = y_train
        batch_loss = train_step(x, y)
        epoch_loss += batch_loss
    
    gradients = optimizer.get_gradients(
        epoch_loss, model.trainable_weights)
    print('Gradients:', gradients)

ValueError: in user code:

    File "/tmp/ipykernel_12070/2399678576.py", line 19, in train_step  *
        y_pred = model(x)
    File "/home/matthijs/anaconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/matthijs/anaconda3/lib/python3.9/site-packages/keras/engine/input_spec.py", line 250, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when calling layer "sequential_13" "                 f"(type Sequential).
    
    Input 0 of layer "dense_26" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (2352,)
    
    Call arguments received by layer "sequential_13" "                 f"(type Sequential):
      • inputs=tf.Tensor(shape=(2352,), dtype=float32)
      • training=None
      • mask=None


In [25]:
import tensorflow as tf
import numpy as np

# define model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train_flatten.shape[1],)),
    tf.keras.layers.Dense(5, activation='softmax')
])

# define optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.metrics.sparse_categorical_crossentropy()

# define training loop
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        # compute model predictions and loss
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
        
    # compute gradients
    gradients = tape.gradient(loss, model.trainable_weights)
    
    # update model parameters
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    return loss

# split data into batches
batch_size = 294
num_batches = len(X_train_flatten) // batch_size
X_batches = np.array_split(X_train_flatten, num_batches)
y_batches = np.array_split(y_train, num_batches)

# train the model and get gradients after each epoch
for epoch in range(10):
    epoch_loss = 0.0
    for i in range(num_batches):
        x_batch = X_batches[i]
        y_batch = y_batches[i]
        batch_loss = train_step(x_batch, y_batch)
        epoch_loss += batch_loss
    
    gradients = optimizer.get_gradients(
        epoch_loss, model.trainable_weights)
    print('Gradients:', gradients)


TypeError: Missing required positional argument

In [56]:
import tensorflow as tf
import numpy as np

# define model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train_flatten.shape[1],)),
    tf.keras.layers.Dense(5, activation='softmax')
])

# define optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.sparse_categorical_crossentropy

# define training loop
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        # compute model predictions and loss
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
        
    # compute gradients
    gradients = tape.gradient(loss, model.trainable_weights)
    
    # update model parameters
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    return loss

# split data into batches
batch_size = 256
num_batches = len(X_train_flatten) // batch_size
X_batches = np.array_split(X_train_flatten, num_batches).astype(float)
y_batches = np.array_split(y_train, num_batches).astype(float)

# train the model and get gradients after each epoch
for epoch in range(10):
    epoch_loss = 0.0
    for i in range(num_batches):
        x_batch = X_batches[i]
        y_batch = y_batches[i]
        batch_loss = train_step(x_batch, y_batch)
        epoch_loss += batch_loss
    try:
        gradients = optimizer.get_gradients(
            epoch_loss, model.trainable_weights)
        print('Gradients:', gradients)
    except:
        print("owh")
    

AttributeError: 'list' object has no attribute 'astype'

In [75]:
import tensorflow as tf
import numpy as np

# define model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train_flatten.shape[1],)),
    tf.keras.layers.Dense(5, activation='relu')
])

# define optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.sparse_categorical_crossentropy

# define training loop
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        # compute model predictions and loss
        
        y_pred = model(x, y)
        print(type(y))
        y = y.astype(np.float32)
        tape.watch(y_pred)
        loss = loss_fn(y, y_pred)
        # y = tf.convert_to_tensor(y, dtype=np.int64)
        
        # y_pred = tf.convert_to_tensor(y_pred, dtype=np.float32)
        
        print(tape.gradient(loss, y_pred))
        
    # compute gradients
    gradients = tape.gradient(loss, model.trainable_weights)
    # # update model parameters
    # optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    return loss

# split data into batches
batch_size = 256
num_batches = len(X_train_flatten) // batch_size
X_batches = np.array_split(X_train_flatten, num_batches)
y_batches = np.array_split(y_train, num_batches)

# train the model and get gradients after each epoch
for epoch in range(10):
    epoch_loss = 0.0
    for i in range(num_batches):
        x_batch = X_batches[i]
        y_batch = y_batches[i]
        train_step(x_batch, y_batch)
        epoch_loss += batch_loss
        gradients = optimizer.get_gradients(epoch_loss, model.trainable_weights)
        # print('Gradients:', gradients)
   
    

<class 'tensorflow.python.framework.ops.Tensor'>


AttributeError: in user code:

    File "/tmp/ipykernel_12070/1287168381.py", line 22, in train_step  *
        y = y.astype(np.float32)

    AttributeError: Tensor object has no attribute 'astype'. 
            If you are looking for numpy-related methods, please run the following:
            from tensorflow.python.ops.numpy_ops import np_config
            np_config.enable_numpy_behavior()
          


In [64]:
x = 10

with tf.GradientTape() as g:
  g.watch(x)
  y = x * x

print(g.gradient(y, x))


ValueError: Passed in object 10 of type 'int', not tf.Tensor or tf.Variable or ExtensionType.